In [ ]:
"""
Script for collecting and preprocessing the raw COVID literature dataset into a format compatible for Mallet
NOTE: only uses the PDF files from the dump on 6/12/2020
"""
import os
path = "/Users/hiancheong/Personal/grad/UMD/comp_ling/topics_evaluation/covid_data_2/document_parses/pdf_json"

all_filenames = []
for (dirname, directories, files) in os.walk(path):
    for fname in files:
        all_filenames.append(f"{dirname}/{fname}")
        

In [3]:
import spacy
nlp = spacy.load("en_core_sci_md")

In [6]:
import json
import scispacy
import spacy

nlp = spacy.load("en_core_sci_md")

open("/Users/hiancheong/Personal/grad/UMD/comp_ling/topics_evaluation/covid_lines.txt", "w+")
file_writer = open("/Users/hiancheong/Personal/grad/UMD/comp_ling/topics_evaluation/covid_lines.txt", "a+")

def valid_token(token):
    if token.is_stop:
        return False
    if token.is_punct:
        return False
    if not token.is_alpha:
        return False
    if "@" in token.text:
        return False

    if token.text:
        return True
    return False

for fname in all_filenames:
    json_doc = json.load(open(fname))
    raw_text = " ".join([section.get("text") for section in json_doc.get("body_text", [])])
    if "the" not in raw_text:
        continue
        
    raw_text = raw_text.lower().replace("\n", " ")
    # limit us to the top 1000000 character
    doc = nlp(raw_text[:999999])
    tokens = [token for token in doc]
    final_text = " ".join([d.text.lower() for d in tokens if valid_token(d)])
    file_writer.write(f"{final_text} \n")


In [ ]:
json_doc = json.load(open(all_filenames[0]))
print([k for k in json_doc.keys()])
print(type(json_doc["body_text"]))
print(json_doc["body_text"][0]["text"])

In [8]:
from langdetect import detect
from collections import defaultdict

open("/Users/hiancheong/Personal/grad/UMD/comp_ling/topics_evaluation/covid_lines_clean_lang.txt", "w+")
file_writer = open("/Users/hiancheong/Personal/grad/UMD/comp_ling/topics_evaluation/covid_lines_clean_lang.txt", "a+")

lang_counts = defaultdict(int)
with open("/Users/hiancheong/Personal/grad/UMD/comp_ling/topics_evaluation/covid_lines.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        lang = detect(line)
        if lang == "en":
            file_writer.write(line)
        lang_counts[lang] += 1
        
print(lang_counts)

defaultdict(<class 'int'>, {'en': 67096, 'de': 742, 'fr': 239, 'nl': 278, 'cy': 8, 'es': 179, 'ca': 12, 'it': 22, 'sq': 1, 'pt': 6, 'ro': 4, 'sv': 1, 'sl': 1, 'af': 1})
